
# Modelo global: predicción del próximo cambio de fresa

Usa `data/processed/fresa_cambios.csv` (una fila por cambio) para predecir el tiempo hasta el siguiente cambio (regresión). Modelo baseline global con split temporal.



## Carga de datos


In [1]:
import pandas as pd
from pathlib import Path

DATA_PATH = Path('../../data/processed/fresa_cambios.csv')

cambios = pd.read_csv(DATA_PATH, parse_dates=['ts_cambio'])
# Asegurar tipos string en IDs
cambios['machine_id'] = cambios['machine_id'].astype(str)
print(cambios.head())
print('Cambios totales:', len(cambios))


  machine_id      machine_name           ts_cambio  piezas_hasta_cambio  \
0         21  Talladora21 LUK3 2025-01-29 08:42:00               9444.0   
1         21  Talladora21 LUK3 2025-01-29 09:26:00                  0.0   
2         21  Talladora21 LUK3 2025-01-31 13:24:00               3250.0   
3         21  Talladora21 LUK3 2025-02-06 11:14:00               3921.0   
4         21  Talladora21 LUK3 2025-02-12 08:53:00               3984.0   

   scrap_hasta_cambio  duracion_cambio_min  ops_hasta_cambio  
0                 8.0                 19.0              37.0  
1                 0.0                 32.0               1.0  
2                 0.0                 39.0              11.0  
3                 6.0                 26.0              16.0  
4                 0.0                 25.0              15.0  
Cambios totales: 702



## Feature engineering y target
- Ordenamos por máquina y tiempo.
- Target: minutos hasta el siguiente cambio en la misma máquina.
- Features simples del ciclo actual: piezas/scrap/ops hasta el cambio, duración del cambio.


In [2]:
cambios = cambios.sort_values(['machine_id', 'ts_cambio']).reset_index(drop=True)

# Target: minutos hasta siguiente cambio por máquina
cambios['next_ts'] = cambios.groupby('machine_id')['ts_cambio'].shift(-1)
cambios['minutes_to_next_change'] = (cambios['next_ts'] - cambios['ts_cambio']).dt.total_seconds() / 60

# Eliminar últimos ciclos sin siguiente cambio
model_df = cambios.dropna(subset=['minutes_to_next_change']).copy()

feature_cols = [
    'piezas_hasta_cambio', 'scrap_hasta_cambio', 'ops_hasta_cambio', 'duracion_cambio_min'
]
X = model_df[feature_cols]
y = model_df['minutes_to_next_change']

print('Dataset modelado:', model_df.shape)
print(model_df[['machine_id','ts_cambio','minutes_to_next_change']].head())


Dataset modelado: (679, 9)
  machine_id           ts_cambio  minutes_to_next_change
0        150 2025-01-09 18:22:00                 19315.0
1        150 2025-01-23 04:17:00                  2213.0
2        150 2025-01-24 17:10:00                 17194.0
3        150 2025-02-05 15:44:00                  8293.0
4        150 2025-02-11 09:57:00                  4227.0



## Split temporal y baseline
Se hace split temporal (80% inicial por tiempo) en todo el conjunto. Modelo: `RandomForestRegressor` (baseline). Métrica: MAE.


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

# Split temporal global
model_df_sorted = model_df.sort_values('ts_cambio')
split_idx = int(len(model_df_sorted) * 0.8)
train_df = model_df_sorted.iloc[:split_idx]
test_df = model_df_sorted.iloc[split_idx:]

X_train = train_df[feature_cols]
y_train = train_df['minutes_to_next_change']
X_test = test_df[feature_cols]
y_test = test_df['minutes_to_next_change']

model = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

preds = model.predict(X_test)
mae = mean_absolute_error(y_test, preds)
print(f"Test MAE (minutos): {mae:.1f}")

feature_importances = pd.Series(model.feature_importances_, index=feature_cols).sort_values(ascending=False)
feature_importances


Test MAE (minutos): 10422.0


piezas_hasta_cambio    0.311357
duracion_cambio_min    0.292414
ops_hasta_cambio       0.229557
scrap_hasta_cambio     0.166673
dtype: float64


## Ejemplo de predicción
Predicción para los últimos cambios de una máquina (ordenados por fecha) usando el modelo entrenado.


In [4]:
example_machine = model_df['machine_id'].iloc[0]
sub = model_df[model_df['machine_id'] == example_machine].sort_values('ts_cambio')
sub['pred_minutes_to_next'] = model.predict(sub[feature_cols])
sub[['machine_id','ts_cambio','minutes_to_next_change','pred_minutes_to_next']].tail(5)


,machine_id,ts_cambio,minutes_to_next_change,pred_minutes_to_next
18,150,2025-06-25 10:10:00,11816.0,8825.06
19,150,2025-07-03 15:06:00,11381.0,14294.66
20,150,2025-07-11 12:47:00,25886.0,11146.03
21,150,2025-07-29 12:13:00,12648.0,12388.70
22,150,2025-08-07 07:01:00,77.0,11048.72



## Próximos pasos
- Añadir features dinámicas: tendencia de piezas (rolling mean de los últimos 2-3 ciclos), ratio scrap/piezas, tiempo real entre cambios previo.
- Evaluar modelos más robustos (XGBoost/LightGBM) y calibrar con split por máquina para evitar fuga temporal.
- Experimentar con clasificación "¿cambio en próximas 8h/12h?" para alertas.
